In [106]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.parse import urljoin
import requests

In [1]:
# Pulls HTML from biopharmcatalyst.com stock listings

def pull_html(ticker):
    try:
        
        r = requests.get("https://www.biopharmcatalyst.com/company/" + ticker
                    , headers={'User-Agent': 'Chrome/67.0.3396.99'})
        
    except Exception :
        print("No internet connection detected")
        return

    if r.status_code == 200:
        r = Request("https://www.biopharmcatalyst.com/company/" + ticker
                    , headers={'User-Agent': 'Chrome/67.0.3396.99'})

        u = urlopen(r)
        html = u.read().decode('utf-8')
        return html
    else:
        print("An HTTP Error has Occured with pull_html(" + ticker + ") Error Code: "+str(r.status_code))
        


In [12]:
def get_dates(html, ticker):
    drug_date_list = []
 
    if html == None:
        return [ticker, "no data"]
    
    elif "Approved" in html:

        index = 1
        while len(html) > 0:
            
            i = html.find(">Approved") 
            if i == -1:
                return drug_date_list
            else:
                date = html[i+10: i+50].split(".<")[0]
                date = date.split("-")[0]
                date = date.split(".")[0]
                date = date.rstrip()
                drug_name = ticker + "_" + "drug_" + str(index)
           
            drug_date_list.append([drug_name, date])
            
            html = html[i+20:]
            
            index += 1
            
    else:
        return [ticker, "none"]
            

In [110]:
import os
import pandas as pd
import numpy as np
rootdir = '/Users/j/Desktop/algo_trading/'

df_tickers = pd.read_excel(rootdir + "tickers.xlsx")

In [113]:
df_tickers 

,S.No.,Company Name,Ticker
0,1,Abeona Therapeutics Inc.,ABEO
1,2,AC Immune SA,ACIU
2,3,ACADIA Pharmaceuticals Inc.,ACAD
3,4,"Acasti Pharma, Inc.",ACST
4,5,"AcelRx Pharmaceuticals, Inc.",ACRX
5,6,Acer Therapeutics Inc.,ACER
6,7,"Achaogen, Inc.",AKAO
7,8,"Achillion Pharmaceuticals, Inc.",ACHN
8,9,"Aclaris Therapeutics, Inc.",ACRS
9,10,"Adamas Pharmaceuticals, Inc.",ADMS


In [38]:
ticker_list = df_tickers["Ticker"].values.tolist()
firm_list = df_tickers["Company Name"].values.tolist()

In [109]:
ticker_list_short = ticker_list[0:100] 

In [95]:
def pull_all_approved_trials(ticker_list):
    approved_trial_list = []
    for ticker in ticker_list:
        print(ticker," ", ticker_list.index(ticker))
        html = pull_html(ticker)
        dates = get_dates(html, ticker)
        try:
            if isinstance(dates[0], list):   
                approved_trial_list = approved_trial_list + dates
            else:
                approved_trial_list.append(dates)
                
        except IndexError:
                approved_trial_list.append(dates)
        
    return approved_trial_list
        

In [108]:

approved_trial_list = pull_all_approved_trials(ticker_list_short)

ABEO   0
ACIU   1


KeyboardInterrupt: 

In [102]:
approved_trial_list_cleaned = [x for x in approved_trial_list if x != []]

In [103]:
def write_dates_excel(lst):
    df_output = pd.DataFrame([x[0] for x in lst])
    df_output["Date"] = [x[1] for x in lst]
    df_output.columns = ["Ticker", "Approved Drug Date"]
    
    writer = pd.ExcelWriter('/Users/j/Desktop/algo_trading/excel_output/pharma_dates_' + str(len(df_output)) + '.xlsx')
    df_output.to_excel(writer,'Sheet1')
    writer.save()
    
    return df_output

In [105]:
write_dates_excel(approved_trial_list_cleaned)

,Ticker,Date
0,ABEO,none
1,ACIU,none
2,ACAD_drug_1,April 29 2016
3,ACRX,none
4,ACER,no data
5,AKAO,none
6,ACHN,none
7,ACRS,none
8,ADMS_drug_1,"December, 24 2014"
9,ADMP,none
